## Google Drive Setup!.

In [14]:
# from google.colab import drive
# drive.mount('/content/drive')

In [15]:
# cd drive/

In [16]:
# cd MyDrive/

In [17]:
# cd MachineLearning/

In [18]:
# cd Data/

In [19]:
# cd ASR challenge/

In [20]:
# ls

## LIBRARIES 

In [21]:
#!pip install jiwer

In [22]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from IPython import display
from jiwer import wer

In [23]:
# import os
# import pickle
# import librosa
# import pandas as pd
# import warnings
# warnings.filterwarnings('ignore', category=DeprecationWarning)
# warnings.filterwarnings('ignore', category=FutureWarning)

# from IPython.display import Audio

# import librosa.display
# import matplotlib.pyplot as plt
# import numpy as np
# import tensorflow as tf

# from tensorflow.keras.models import Model, Sequential
# from tensorflow.keras.layers import * 
# from tensorflow.keras.callbacks import Callback, EarlyStopping, ModelCheckpoint, TensorBoard
# from tensorflow.keras.preprocessing.sequence import pad_sequences
# from tensorflow.keras.optimizers import SGD, Adam, RMSprop
# from tensorflow.keras import backend as K

# from tensorflow import keras
# from tensorflow.keras import layers
# from IPython import display
# from jiwer import wer

## Csv data and Audios.

In [24]:
# ## Converting mp3 to wavs files.

# from pydub import AudioSegment
# import os

# # Set the paths for input MP3 files and output WAV files.
# input_folder = "/data/audio_files/train_audios/"
# output_folder = "train_wavs/"

# # Ensure the output folder exists, create it if not.
# os.makedirs(output_folder, exist_ok=True)

# # Convert MP3 to WAV function
# def convert_mp3_to_wav(mp3_path, wav_path):
#     sound = AudioSegment.from_mp3(mp3_path)
#     sound.export(wav_path, format="wav")

# # Iterate through MP3 files in the input folder and convert to WAV.
# for mp3_file in os.listdir(input_folder):
#     if mp3_file.endswith(".mp3"):
#         mp3_path = os.path.join(input_folder, mp3_file)
#         wav_file = os.path.splitext(mp3_file)[0] + ".wav"
#         wav_path = os.path.join(output_folder, wav_file)
#         convert_mp3_to_wav(mp3_path, wav_path)
#         print(f"Converted {mp3_file} to {wav_file}")

# print("Conversion completed!.")


# validation and training data.

In [25]:
wavs_path = 'data/audio_files/train_audios/'
metadata_path = 'data/train.csv'

In [ ]:
metadata_df = pd.read_csv(metadata_path)
metadata_df.head()

In [ ]:
# metadata_df.columns = ["file_name", "transcription", "normalized_transcription"]
metadata_df = metadata_df[["audio_ID", "sentence"]]
# metadata_df = metadata_df.sample(frac=1).reset_index(drop=True)
metadata_df.head()

,audio_ID,sentence
0,audio_faa7312a,huko kwa Wakiroba Mkoa wa Mara
1,audio_643a10c1,Alingaa katika medani za kisiasa na uongozi nd...
2,audio_5b626e74,Vitu saba ambavyo kila baba atakuwa.
3,audio_5972c5f3,inaonyesha mawaziri wapya ambao wamechukua naf...
4,audio_deebd5b0,ee hii pia inatumiwa na kiwanda cha


In [ ]:
metadata_df.shape

(19093, 2)

In [ ]:
# Reducing the size of data used to reduce training time and automatically create new dataframe.

metadata_df.head(1200).to_csv('data/notebook - 1.csv', index=False)

In [ ]:
metadata_df = pd.read_csv('data/notebook - 1.csv')


In [ ]:
# splitting into training and validation data.

split = int(len(metadata_df) * 0.90 )
# split2 = int(len(metadata_df) * 0.2 )
train_df = metadata_df[:split]
df_val = metadata_df[split:]

print(f"Size of training set: {len(train_df)}")
print(f"Size of validation set: {len(df_val)}")

Size of training set: 160
Size of validation set: 40


In [ ]:
train_df.head()

,audio_ID,sentence
0,audio_faa7312a,huko kwa Wakiroba Mkoa wa Mara
1,audio_643a10c1,Alingaa katika medani za kisiasa na uongozi nd...
2,audio_5b626e74,Vitu saba ambavyo kila baba atakuwa.
3,audio_5972c5f3,inaonyesha mawaziri wapya ambao wamechukua naf...
4,audio_deebd5b0,ee hii pia inatumiwa na kiwanda cha


In [ ]:
df_val.head()

,audio_ID,sentence
160,audio_e8c36596,hasa barani afrika
161,audio_cfd5f18f,mwandishi wetu zuhra mwera ametuandalia taarif...
162,audio_26d8c981,Wakristo wakiwemo Waprotestanti na Wakatoliki ...
163,audio_b4cca469,Serikali imebainisha majukumu ya mamlaka mbali...
164,audio_ce851761,Nyumba imejengwa.


## PREPROCESSING:

##### change characters to integers and integers to charaters.

In [ ]:
# The set of characters accepted in the sentence
characters = [x for x in "abcdefghijklmnopqrstuvwxyz'?! "]

# mapping characters to integers.
char_to_num = keras.layers.StringLookup(vocabulary=characters, oov_token='')
# Mapping integers back to original characters.
num_to_char = keras.layers.StringLookup(
    vocabulary=char_to_num.get_vocabulary(), oov_token="", invert=True
)

print(
    f"The vocabulary is : {char_to_num.get_vocabulary()}"
    f"(size is : {char_to_num.vocabulary_size()})"
)

The vocabulary is : ['', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', "'", '?', '!', ' '](size is : 31)


In [ ]:
# An integer scalar Tensor. The window length in samples.
frame_length = 256
# An integer scalar Tensor. The number of samples to step.
frame_step = 160
# An integer scalar Tensor , The size of the FFT to apply.
# If not provided , uses the smallest power of 2 enclosing frame lenght.
fft_length = 384

def encode_single_sample(wav_file, label):
    ## Process the audio.
    ## Read the audio file.
    file = tf.io.read_file(wavs_path + wav_file + ".wav")

    ## Decode the audio file
    audio, _ = tf.audio.decode_wav(file)
    audio = tf.squeeze(audio, axis=-1)
    ## Change type to float.
    audio = tf.cast(audio, tf.float32)
    ## Get the spectrogram
    spectrogram = tf.signal.stft(
        audio, frame_length=frame_length, frame_step=frame_step, fft_length=fft_length
    )

    ## We only need the magnitude which can be derived by applying tf.abs
    spectrogram = tf.abs(spectrogram)
    spectrogram = tf.math.pow(spectrogram, 0.5)

    ## normalization.
    means = tf.math.reduce_mean(spectrogram, 1, keepdims=True)
    stddevs = tf.math.reduce_std(spectrogram, 1, keepdims=True)
    spectrogram = (spectrogram - means) / (stddevs + 1e-10)

    ## Process the label
    ## Convert the label to lower case.
    label = tf.strings.lower(label)
    ## split the label
    label = tf.strings.unicode_split(label, input_encoding='UTF-8')
    ## Map the characters in label to numbers.
    label = char_to_num(label)
    # Return a dict as our model is expecting two inputs.
    return spectrogram, label


In [ ]:
batch_size = 32
# Define the training dataset.
train_dataset = tf.data.Dataset.from_tensor_slices(
    (list(train_df["audio_ID"]), list(train_df["sentence"]))

)

train_dataset = (
    train_dataset.map(encode_single_sample, num_parallel_calls=tf.data.AUTOTUNE)
    .padded_batch(batch_size)
    .prefetch(buffer_size=tf.data.AUTOTUNE)
)

# Define the validation dataset.
validation_dataset = tf.data.Dataset.from_tensor_slices(
    (list(df_val["audio_ID"]), list(df_val["sentence"]))
)

validation_dataset = (
    validation_dataset.map(encode_single_sample, num_parallel_calls=tf.data.AUTOTUNE)
    .padded_batch(batch_size)
    .prefetch(buffer_size=tf.data.AUTOTUNE)
)

### CTC Loss Function.

##### Function for time and loss function.

In [ ]:
def CTCLoss(y_true, y_pred):
    # Compute the training-time loss_value
    
    batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64")
    input_length = tf.cast(tf.shape(y_pred)[1], dtype="int64")
    label_length = tf.cast(tf.shape(y_true)[1], dtype="int64")

    input_length = input_length * tf.ones(shape=(batch_len, 1), dtype='int64')
    label_length = label_length * tf.ones(shape=(batch_len, 1), dtype='int64')

    loss = keras.backend.ctc_batch_cost(y_true, y_pred, input_length, label_length)

    return loss

### Model Building.

In [ ]:
def build_model(input_dim, output_dim, rnn_layers=5, rnn_units=128):
    
    ## Model similar to DeepSpeech2.
    input_spectrogram = layers.Input((None, input_dim), name="input")
    ## Expand the dimension to use 2D CNN
    x = layers.Reshape((-1, input_dim, 1), name="expand_dim")(input_spectrogram)

    ## Convolution layer 1
    x = layers.Conv2D(
        filters=32,
        kernel_size=[11,41],
        strides=[2, 2],
        padding="same",
        use_bias=False,
        name="conv_1",
    )(x)
    x = layers.BatchNormalization(name="conv_1_bn")(x)
    x = layers.ReLU(name="conv_1_relu")(x)

    ## Convolution layer 2
    x = layers.Conv2D(
        filters=32,
        kernel_size=[11, 21],
        strides=[1,2],
        padding="same",
        use_bias=False,
        name="conv_2",
    )(x)
    x = layers.BatchNormalization(name="conv_2_bn")(x)
    x = layers.ReLU(name="conv_2_relu")(x)
    # Reshape the resulted volume to feed the RNNs layers.
    x = layers.Reshape((-1, x.shape[-2] * x.shape[-1]))(x)
    # RNN Layers.
    for i in range(1, rnn_layers + 1):
        recurrent = layers.GRU(
            units=rnn_units,
            activation="tanh",
            recurrent_activation="sigmoid",
            use_bias=True,
            return_sequences=True,
            reset_after=True,
            name=f'gru_{i}',
        )
        x = layers.Bidirectional(
            recurrent, name=f"bidirectional_{i}", merge_mode="concat",
        )(x)
        if i < rnn_layers:
            x = layers.Dropout(rate=0.5)(x)

    # Dense layer.
    x = layers.Dense(units=rnn_units * 2, name="dense_1")(x)
    x = layers.ReLU(name="dense_1_relu")(x)
    x = layers.Dropout(rate=0.5)(x)
    # Classification layer.
    output = layers.Dense(units=output_dim + 1, activation="softmax")(x)
    # Model
    model = keras.Model(input_spectrogram, output, name="DeepSpeech_2")
    # Optimizer.
    opt = keras.optimizers.Adam(learning_rate=1e-4)
    # Compile the model and return
    model.compile(optimizer=opt, loss=CTCLoss)
    return model

# Get the model.
model = build_model(
    input_dim=fft_length // 2 + 1,
    output_dim=char_to_num.vocabulary_size(),
    rnn_units=512,
)
model.summary(line_length=110)

Model: "DeepSpeech_2"
______________________________________________________________________________________________________________
 Layer (type)                                    Output Shape                                Param #          
 input (InputLayer)                              [(None, None, 193)]                         0                
                                                                                                              
 expand_dim (Reshape)                            (None, None, 193, 1)                        0                
                                                                                                              
 conv_1 (Conv2D)                                 (None, None, 97, 32)                        14432            
                                                                                                              
 conv_1_bn (BatchNormalization)                  (None, None, 97, 32)                     

## Training and evaluating.

In [ ]:
# A utility function to decode the output of the network.
def decode_batch_predictions(pred):
    input_len = np.ones(pred.shape[0] * pred.shape[1])
    # Use_greedy search
    results = keras.backend.ctc_decode(pred, input_length=input_len, greedy=True)[0][0]
    ## Iterate over the results and get back the text.
    output_text = []
    for result in results:
        result = tf.strings.reduce_join(num_to_char(result)).numpy().decode('utf-8')
        output_text.append(result)
    return output_text

## A callback class to output a few transcription during training.
class CallbackEval(keras.callbacks.Callback):
    ## Display a batch of outputs after every epoch.

    def __init__(self, dataset):
        super().__init__()
        self.dataset = dataset

    def on_epoch_end(self, epoch: int, logs=None):
        predictions = []
        targets = []
        for batch in self.dataset:
            X, y = batch
            batch_predictions = model.predict(X)
            batch_predictions = decode_batch_predictions(batch_predictions)
            predictions.extend(batch_predictions)
            for label in y:
                label = (
                    tf.strings.reduce_join(num_to_char(label)).numpy().decode('utf-8')
                )
                targets.append(label)

        wer_score = wer(targets, predictions)
        print("-" * 100)
        print(f"Word Error Rate: {wer_score:.4f}")
        print("-" * 100)
        for i in np.random.randint(0, len(predictions), 2):
            print(f"Target    : {targets[i]}")
            print(f"Prediction: {predictions[i]}")
            print("-" * 100)

### Training phase

In [ ]:
## Lets start training.
epochs = 1
## CallBack function to check sentence in validation set.
validation_callback = CallbackEval(validation_dataset)
## Train the model.
history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=epochs,
    callbacks=[validation_callback],

    )

## Inferences

In [ ]:
## Let us check the results on the validation samples.
predictions = []
targets = []
for batch in validation_dataset:
    X, y = batch
    batch_predictions = model.predict(X)
    batch_predictions = decode_batch_predictions(batch_predictions)
    predictions.extend(batch_predictions)
    for label in y:
        label = tf.strings.reduce_join(num_to_char(label)).numpy().decode("utf-8")
        targets.append(label)
wer_score = wer(targets, predictions)
print("-" * 100)
print(f"Word Error Rate: {wer_score:.4f}")
print("-" * 100)
for i in np.random.randint(0, len(predictions), 5):
        print(f"Target    : {targets[i]}")
        print(f"Prediction: {predictions[i]}")
        print("-" * 100)